In [1]:
import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [2]:
#Checks if current position is not floating.
def CheckBelow(Environment, X, Y, Z):
    return Z == 0 or (Environment[X,Y,:Z] != '0').all()

#Checks if current position has container.
def CheckPos(Environment, Size, X, Y, Z):
    if not ExceedsLot(Size, X, Y, Z):
        return Environment[X, Y, Z] != '0'
    else:
        return False

#Checks if coordinates exceed lot.
def ExceedsLot(Size, X, Y, Z):
    return Size[2] <= Z or Size[1] <= Y or Size[0] <= X or Z < 0 or Y < 0 or X < 0

#PAS AAN, bij 4 of 5 breed kan er nog een container tussen gezet worden
#Check whether container is in the middle of other containers.
def CheckMiddle(Environment, Size, X, Y, Z):
    return CheckPos(Environment, Size, X, Y - 1, 0) and CheckPos(Environment, Size, X, Y + 1, 0)

#Adds container to environment using state.
def AddContainer(Environment, Size, State, X, Y, Z):
    tempEnvironment = np.copy(Environment)
    if CheckBelow(Environment, X, Y, Z) and not CheckPos(Environment, Size, X, Y, Z) and not ExceedsLot(Size, X, Y, Z) and not CheckMiddle(Environment, Size, X, Y, Z):
        tempEnvironment[X,Y,Z] = State
    return tempEnvironment

In [3]:
def scoreEnvironment(Environment):
    result = 0
    for x in range(Environment.shape[0]):
        for y in range(Environment.shape[1]):
            for z in range(Environment.shape[2]):
                result = (Environment[0,:,:] == 'L').sum()
                result -= (Environment[0,:,:] == 'H').sum()
                result -= (Environment[2,:,:] == 'L').sum()
                result += (Environment[2,:,:] == 'H').sum()
    return result


#Returns cleared environment
def clearEnvironment(Environment):
    return np.array([[['0'] * Size[2]] * Size[1]] * Size[0])

#Generate status of board in Boolean array
def GenerateStatusDataset(Environment,dataType):
    return np.concatenate([np.array([dataType == 'H'],dtype=np.float32),np.array(Environment == 'L',dtype=np.float32).flatten(), np.array(Environment == 'H', dtype=np.float32).flatten(), np.array(Environment == '0',dtype=np.float32).flatten()])

In [4]:
def generateEnvironment(size=[3,3,2]):
    c =  [[['0'] * size[2]] * size[1]] * size[0]
    cSize = size[0]*size[1]*size[2]*3
    actionSize = size[0]*size[1]*size[2]
    speelveld = np.array(c)
    return speelveld, size

In [5]:
generateEnvironment()

(array([[['0', '0'],
         ['0', '0'],
         ['0', '0']],
 
        [['0', '0'],
         ['0', '0'],
         ['0', '0']],
 
        [['0', '0'],
         ['0', '0'],
         ['0', '0']]], dtype='<U1'),
 [3, 3, 2])

In [2]:
def coord_to_ind(X,Y,Z,lot_size):
    return np.ravel_multi_index((X,Y,Z), lot_size)

#Zet de index in array om naar X,Y,Z coördinaten
def ind_to_coord(ind, lot_size):
    return np.unravel_index(ind,lot_size)

In [19]:
lotsize = [4,4,5]
def vulenv(lot, index, prio):
    act_loc = ind_to_coord(index, lotsize)
    new_lot = AddContainer(lot, lotsize, prio, act_loc[0], act_loc[1], act_loc[2])
    return new_lot

lot = generateEnvironment(lotsize)[0]

lot = vulenv(lot, 0, 1)
lot = vulenv(lot, 1, 2)
lot = vulenv(lot, 2, 1)
lot = vulenv(lot, 3, 1)
lot = vulenv(lot, 4, 2)
lot = vulenv(lot, 5, 2)
lot = vulenv(lot, 6, 3)
lot = vulenv(lot, 7, 3)
lot = vulenv(lot, 8, 3)
lot = vulenv(lot, 9, 3)
lot = vulenv(lot, 10, 2)
lot = vulenv(lot, 11, 1)
lot = vulenv(lot, 12, 1)
lot = vulenv(lot, 13, 1)
lot = vulenv(lot, 14, 2)
lot = vulenv(lot, 15, 2)
lot = vulenv(lot, 16, 3)
lot = vulenv(lot, 17, 3)
print(lot)

idc = 0
rs = cont_routes(lot, lotsize,idc)
print('container: {0}, routes: {1}'.format(idc, rs))
print('-----------------')
r1 = rs[0]
r2 = rs[1]
#print(route_info(r1))
#print(route_info(r2))
print('score: {0}'.format(score_eindveld(lot,lotsize)))

NameError: name 'generateEnvironment' is not defined

In [64]:
#Geeft van een gegeven containerindex in de lotindeling de twee wegen om deze container te bereiken
def cont_routes(lot_indeling, lot_size, ind): #lot_indeling: array, lot_size: [x,y,z], ind: int
    X = lot_size[0]
    Y = lot_size[1]
    Z = lot_size[2]
    lot_index = np.arange(np.prod(lot_size)).reshape(lot_size) #lotindeling maar dan de indexes
    indc = ind_to_coord(ind, lot_size) #vind coördinaten van ingevoerde index
    
    #voeg de containerstack waar de betreffende container in zit toe aan de dictionary als start van de route
    stackvalue = lot_indeling[indc[0]][indc[1]]
    stackkey = lot_index[indc[0]][indc[1]]
    stackdict = dict(zip(stackkey,stackvalue))
    
    #route 1 berekenen
    routestart = {k: v for k, v in stackdict.items() if k >= ind}
    route1 = routestart
    i = 0
    while (lot_index[indc[0]][indc[1]-i][0] % (Y*Z)) > 0:
        for z in range(Z):
            route1[lot_index[indc[0]][indc[1]-(i+1)][z]] = lot_indeling[indc[0]][indc[1]-(i+1)][z]
        i += 1
    
    #route 2 berekenen
    routestart = {k: v for k, v in stackdict.items() if k >= ind}
    route2 = routestart
    i = 0
    while (lot_index[indc[0]][indc[1]+i][Z-1] % (Y*Z)) < (Y*Z-1):
        for z in range(Z):
            route2[lot_index[indc[0]][indc[1]+(i+1)][z]] = lot_indeling[indc[0]][indc[1]+(i+1)][z]
        i += 1
    
    return route1, route2 #route1: dictionary, route2: dictionary


#Geeft de score van een indeling terug
def score_eindveld(lot_indeling):
    X,Y,Z = lot_indeling.shape
    
    lot_index = np.arange(np.prod(lot_indeling.shape)).reshape(lot_indeling.shape)
    
    #maak een lijst van alle verschillende prio indexen in de indeling
    #'0' geeft een lege plek aan, dus die telt niet mee
    #de hoogste waarde (laagste prio) nemen we niet mee
    if np.unique(lot_indeling)[0] == '0':
        lot_unique = np.unique(lot_indeling)[1:-1]
    else:
        lot_unique = np.unique(lot_indeling)[:-1]

    #beloon 0 rijen alleen bij een X >= 1, Y >= 3, Z >= 1
    inbox = 0
    
    for p in lot_unique:
    
        for i in range(len(np.where(lot_indeling == p)[0])):
            obstaclesdict = {}
            x,y,z = np.array(np.where(lot_indeling == p))[:,i]
            
            #x = np.where(lot_indeling == p)[0][i]
            #y = np.where(lot_indeling == p)[1][i]
            #z = np.where(lot_indeling == p)[2][i]
            routes = cont_routes(lot_indeling, lot_indeling.shape, lot_index[x][y][z])
            routelen1 = len({k: v for k,v in routes[0].items() if int(v) > int(p)})
            routelen2 = len({k: v for k,v in routes[1].items() if int(v) > int(p)})

            #kies eerste route als die korter is dan tweede
            if routelen1 < routelen2:
                shortroute = routes[0]
            #kies tweede route als die korter is dan eerste
            elif routelen1 > routelen2:
                shortroute = routes[1]
            #als beide routes even lang zijn, kijk dan hoeveel containers op de route dezelfde prio klasse hebben
            else:
                sameclassroute1 = sum(1 for v in routes[0].values() if v <= list(routes[0].values())[0])
                sameclassroute2 = sum(1 for v in routes[1].values() if v <= list(routes[1].values())[0])
                #kies eerste route als aantal dezelfe prio klasse containers gelijk aan of meer is dan tweede
                if sameclassroute1 >= sameclassroute2:
                    shortroute = routes[0]
                #kies tweede route als aantal dezelfe prio klasse containers meer is dan eerste
                else:
                    shortroute = routes[1]
            
            #sla alle containers op die op de route in de weg zitten (de containers met een hogere prioklasse)
            for c in shortroute:
                if int(shortroute[c]) > int(p):
                    obstaclesdict[c] = shortroute[c]
            
            #score omlaag als containers in de weg zitten
            if len(obstaclesdict) > 0:
                inbox += 1

    
    return inbox #score: integer

In [178]:
def getRowScore(row, height, priority):
    side = row[1:] <= priority
    above = row[0][height:] <= priority
    total = side.sum() + above.sum()
    totalcontainers = side.size+above.size
    return total/totalcontainers
    
def getContainerMovementScore(row,pos,prio):
    right=row[pos[0]:,:]
    left=np.flip(row[:pos[0]+1,:],axis=0)
    return min(getRowScore(left,pos[1],prio),getRowScore(right,pos[1],prio))

def rowScore(env):
    totalScore = 0
    #Per row
    for row in env:
        it = np.nditer(row, flags=['multi_index'])
        for container in it:
            totalScore += getContainerMovementScore(row,it.multi_index,container)

    return totalScore

In [184]:
np.random.seed(1)
env = np.random.randint(1,4,size=(1,2,2))
print(rowScore(env))
env

3.1666666666666665


array([[[2, 1],
        [1, 2]]])